# vLLM baichuan2 rollingbatch deployment guide
In this tutorial, you will use LMI container from DLC to SageMaker and run inference with it.

Please make sure the following permission granted before running the notebook:

- S3 bucket push access
- SageMaker access

## Step 1: Let's bump up SageMaker and import stuff

In [1]:
import boto3
import sagemaker
from sagemaker import Model, image_uris, serializers, deserializers

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
region = sess._region_name  # region name of the current SageMaker Studio environment
account_id = sess.account_id()  # account_id of the current SageMaker Studio environment
bucket = sess.default_bucket()
s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
!pip install huggingface_hub

In [10]:
from huggingface_hub import snapshot_download
from pathlib import Path


local_cache_path = Path("./model")
local_cache_path.mkdir(exist_ok=True)

model_name = "baichuan-inc/Baichuan2-13B-Chat"

# Only download pytorch checkpoint files
allow_patterns = ["*.json", "*.pt", "*.bin", "*.model", "*.py", "*.txt"]

model_download_path = snapshot_download(
    repo_id=model_name,
    cache_dir=local_cache_path,
    allow_patterns=allow_patterns,
    #revision='f2cc3a689c5eba7dc7fd3757d0175d312d167604'
)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

configuration_baichuan.py:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

generation_utils.py:   0%|          | 0.00/2.97k [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/285 [00:00<?, ?B/s]

...

quantizer.py:   0%|          | 0.00/9.11k [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/23.3k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/544 [00:00<?, ?B/s]

tokenization_baichuan.py:   0%|          | 0.00/9.03k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/954 [00:00<?, ?B/s]

In [15]:
# Get the model files path
import os
from glob import glob

local_model_path = None

paths = os.walk(r'./model')

for root, dirs, files in paths:
    for file in files:
        if file == 'config.json':
            print(os.path.join(root,file))
            local_model_path = str(os.path.join(root,file))[0:-11]
            print(local_model_path)
if local_model_path == None:
    print("Model download may failed, please check prior step!")

./model/models--baichuan-inc--Baichuan2-13B-Chat/snapshots/8f6e343d545c503b91429582231d1d354dac2740/config.json
./model/models--baichuan-inc--Baichuan2-13B-Chat/snapshots/8f6e343d545c503b91429582231d1d354dac2740/


In [ ]:
!wget https://github.com/peak/s5cmd/releases/download/v2.1.0/s5cmd_2.1.0_Linux-64bit.tar.gz
!tar xvzf s5cmd_2.1.0_Linux-64bit.tar.gz

In [18]:
!chmod +x ./s5cmd
!./s5cmd sync ./model/models--baichuan-inc--Baichuan2-13B-Chat/snapshots/8f6e343d545c503b91429582231d1d354dac2740/ s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_baichuan2_model/

cp model/models--baichuan-inc--Baichuan2-13B-Chat/snapshots/8f6e343d545c503b91429582231d1d354dac2740/special_tokens_map.json s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_baichuan2_model/special_tokens_map.json
cp model/models--baichuan-inc--Baichuan2-13B-Chat/snapshots/8f6e343d545c503b91429582231d1d354dac2740/tokenizer_config.json s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_baichuan2_model/tokenizer_config.json
cp model/models--baichuan-inc--Baichuan2-13B-Chat/snapshots/8f6e343d545c503b91429582231d1d354dac2740/handler.py s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_baichuan2_model/handler.py
cp model/models--baichuan-inc--Baichuan2-13B-Chat/snapshots/8f6e343d545c503b91429582231d1d354dac2740/pytorch_model.bin.index.json s3://sagemaker-us-west-2-687912291502/LLM-RAG/workshop/LLM_baichuan2_model/pytorch_model.bin.index.json
cp model/models--baichuan-inc--Baichuan2-13B-Chat/snapshots/8f6e343d545c503b91429582231d1d354dac2740/modeling_baichuan.py s3

## Step 2: Start preparing model artifacts
In LMI contianer, we expect some artifacts to help setting up the model
- serving.properties (required): Defines the model server settings
- model.py (optional): A python file to define the core inference logic
- requirements.txt (optional): Any additional pip wheel need to install

In [19]:
%%writefile serving.properties
engine=Python
option.model_id=baichuan-inc/Baichuan2-13B-Chat
option.task=text-generation
option.trust_remote_code=true
option.tensor_parallel_degree=4
option.rolling_batch=vllm
option.dtype=fp16
option.enable_streaming=true

Writing serving.properties


In [20]:
%%writefile requirements.txt
vllm==0.2.2
pandas
pyarrow
git+https://github.com/huggingface/transformers

Writing requirements.txt


In [21]:
%%sh
mkdir mymodel
mv serving.properties mymodel/
mv requirements.txt mymodel/
tar czvf mymodel.tar.gz mymodel/
rm -rf mymodel

mymodel/
mymodel/serving.properties
mymodel/requirements.txt


## Step 3: Start building SageMaker endpoint
In this step, we will build SageMaker endpoint from scratch

### Getting the container image URI

[Large Model Inference available DLC](https://github.com/aws/deep-learning-containers/blob/master/available_images.md#large-model-inference-containers)


In [22]:
image_uri = image_uris.retrieve(
        framework="djl-deepspeed",
        region=sess.boto_session.region_name,
        version="0.25.0"
    )

### Upload artifact on S3 and create SageMaker model

In [23]:
s3_code_prefix = "large-model-lmi/code"
bucket = sess.default_bucket()  # bucket to house artifacts
code_artifact = sess.upload_data("mymodel.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {code_artifact}")

model = Model(image_uri=image_uri, model_data=code_artifact, role=role)

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-west-2-687912291502/large-model-lmi/code/mymodel.tar.gz
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


### 4.2 Create SageMaker endpoint

You need to specify the instance to use and endpoint names

In [ ]:
instance_type = "ml.g5.12xlarge"
endpoint_name = sagemaker.utils.name_from_base("vllm-baichuan2")

model.deploy(initial_instance_count=1,
             instance_type=instance_type,
             endpoint_name=endpoint_name,
             # container_startup_health_check_timeout=3600
            )

# our requests and responses will be in json format so we specify the serializer and the deserializer
predictor = sagemaker.Predictor(
    endpoint_name=endpoint_name,
    sagemaker_session=sess,
    serializer=serializers.JSONSerializer(),
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
------------------------------------

## Step 5: Test and benchmark the inference

In [ ]:
import io

def extract_unicode_chars(text):
    pattern = r'\\u([\dA-Fa-f]{4})'
    result = re.sub(pattern, lambda m: chr(int(m.group(1), 16)), text)
    return result


class StreamScanner:
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0
        
        
class LineIterator:
    def __init__(self, stream):
        self.byte_iterator = iter(stream)
        self.buffer = io.BytesIO()
        self.read_pos = 0

    def __iter__(self):
        return self

    def __next__(self):
        while True:
            self.buffer.seek(self.read_pos)
            line = self.buffer.readline()
            print(line)
            if line and line[-1] == ord('\n'):
                self.read_pos += len(line)
                return line[:-1]
            try:
                chunk = next(self.byte_iterator)
            except StopIteration:
                if self.read_pos < self.buffer.getbuffer().nbytes:
                    continue
                raise
            if 'PayloadPart' not in chunk:
                print('Unknown event type:' + chunk)
                continue
            self.buffer.seek(0, io.SEEK_END)
            self.buffer.write(chunk['PayloadPart']['Bytes'])

In [ ]:
parameters = {
                "max_tokens": 300,
                #"do_sample": True,
                "temperature": 0.5
            }

prompt_live="""Create a captivating product promotion script for an e-commerce live broadcast host based on the following product detailed description.
[The iPhone 14 is the latest phone officially released by Apple on September 8, 2022. It is equipped with a 6.1-inch OLED screen and offers six unique color choices: blue, purple, midnight, starlight, red, and yellow. The phone has an elegant size design, with a length of 146.7mm, a width of 71.5mm, a thickness of 7.8mm, and weighs about 172 grams. In terms of performance, the iPhone 14 is equipped with a powerful Apple A15 bionic chip, which contains a 6-core central processor, including 2 performance cores and 4 efficiency cores, and a 5-core GPU graphics processor. It not only supports practical features such as accident detection and satellite communication but also performs excellently in photography. The rear camera includes a 12-megapixel main lens and a 12-megapixel ultra-wide-angle lens, and the front camera is also 12 megapixels. In addition, the phone also supports photographic technologies such as optical engine, deep fusion technology, smart HDR 4, and portrait mode, ensuring that users can easily capture every beautiful moment.]
The script should include the main features and advantages of the product as well as interactive segments, be written in Chinese, and be concise, interesting, and attractive, ensuring to contain all the elements mentioned above.
"""
prompt1 = """根据以下反引号内的商品详细描述，为电商直播主持人创作一段引人注目的商品推介话术
‘’‘
iPhone 14是苹果公司在2022年9月8日正式发布的最新手机。它配备了一块6.1英寸的OLED屏幕，并提供了六种独特的颜色选择：蓝色、紫色、午夜色、星光色、红色和黄色。手机的尺寸设计优雅，长度为146.7毫米，宽度为71.5毫米，厚度为7.8毫米，重量约为172克。
在性能上，iPhone 14搭载了强大的苹果A15仿生芯片，内部含有6 核中央处理器，有 2 个性能核心和 4 个能效核心，还有5 核GPU图形处理器。。它不仅支持车祸检测和卫星通信等实用功能，而且在拍照方面也表现出色。
后置摄像头包括一个1200万像素的主镜头和一个1200万像素的超广角镜头，前置摄像头也是1200万像素
此外，该手机还支持光像引擎、深度融合技术、智能HDR4和人像模式等摄影技术，确保用户可以轻松捕捉每一个美好瞬间
’‘’
话术中应包括商品的主要特点、优势及互动环节,使用中文撰写，并保持话术简洁、有趣且具吸引力,并确保包含上述要求的所有元素"""

prompt2="""根据以下反引号内的关键词，为电商直播主持人创作一段通用的开场、互动或欢迎话术。请确保话术融入这些关键词，使用中文撰写，内容要简洁、有趣且具吸引力，同时适应广泛的商品和场景。
‘’‘
精选
性价比
品质
日常家居
穿搭
限时折扣
免费赠品
抽奖活动
大品牌合作
独家优惠
’‘’
请使用上述关键词，编写一段具有普遍适用性，适于电商直播开头或互动环节的话术"""


prompt3="""
请根据以下反引号内的商品描述、意图、问题模板和回答模板，为电商直播商品生成一个问答库。要求生成的回答应当有至少一组，最多五组。请确保答案基于商品描述和回答模板生成。如果无法生成回答，表示为“根据已知信息无法生成回答”。格式应如下：{{"Q":"问题","A":['答案1-1','答案1-2'...]}}
‘’‘
商品描述：iPhone 14是苹果公司在2022年9月8日正式发布的最新手机。它配备了一块6.1英寸的OLED屏幕，并提供了六种独特的颜色选择：蓝色、紫色、午夜色、星光色、红色和黄色。手机的尺寸设计优雅，长度为146.7毫米，宽度为71.5毫米，厚度为7.8毫米，重量约为172克。
在性能上，iPhone 14搭载了强大的苹果A15仿生芯片，内部含有6 核中央处理器，有 2 个性能核心和 4 个能效核心，还有5 核GPU图形处理器。。它不仅支持车祸检测和卫星通信等实用功能，而且在拍照方面也表现出色。后置摄像头包括一个1200万像素的主镜头和一个1200万像素的超广角镜头，前置摄像头也是1200万像素。此外，该手机还支持光像引擎、深度融合技术、智能HDR4和人像模式等摄影技术，确保用户可以轻松捕捉每一个美好瞬间。}
意图：性能
问题模板：手机的性能如何？
回答模板：[商品名称]采用了最新的[芯片名称]，搭载了[核心数量]核CPU和[GPU核心数量]核GPU，为用户提供强大的性能。
谈到[商品名称]的性能，不得不提及它的[芯片名称]，配备[核心数量]核CPU和[GPU核心数量]核GPU，应对各种任务都游刃有余。
[商品名称]在性能上表现卓越，得益于其[芯片名称]和[核心数量]核处理器，加上[GPU核心数量]核GPU，让每次使用都顺畅无比。}
’‘’
问答生成：请基于上述商品描述、意图、问题模板和回答模板，为电商直播商品提供符合上述格式的问答库。
"""

prompt4="""
请以电商直播主持人的第一人称角度回答观众的商品相关问题。确保只回答与商品相关的问题，并只使用以下反引号内知识库的信息来回答。回答中请勿随意编造内容。格式应如下:[{{"intention": "意图1", "answer": "回答1"}},{{"intention": "意图2", "answer": "回答2"}}]
‘’‘
[问题：iPhone 14有哪些可选的颜色？][回答：iPhone 14提供了六种时尚的颜色选择，包括蓝色、紫色、午夜色、星光色、红色和黄色。][意图：颜色]
[问题：关于摄像头，iPhone 14的前置和后置摄像头分辨率是多少？][回答：iPhone 14的前置和后置摄像头分辨率都是1200万像素。][意图：分辨率]
[问题：我经常用手机办公和玩游戏，iPhone 14的性能如何？][回答：iPhone 14搭载了强大的苹果A15六核中央处理器，无论是玩游戏、看视频，还是办公，它都可以轻松应对。][意图：性能]}
’‘’
观众问题：主播小姐姐好漂亮
使用第一人称直接回答观众关于商品的提问。检查知识库中是否有与观众提问相匹配的回答。对于在知识库中找到的每个匹配意图，请依次提供对应的回答，并确保从知识库中的意图中提取相应的意图标签。如果所有的意图都在知识库中找不到答案，回答“根据已知信息无法回答问题”。确保不使用emoji。
"""

other="""我很在意手机的颜色和摄像头功能，能给我介绍一下iPhone 14在这两方面的特点吗？"""


In [ ]:
endpoint_name="vllm-baichuan2-2023-12-01-10-01-07-677"

In [ ]:
%%time
from joblib import Parallel, delayed
import json
import codecs
import re
import datetime

prompts = [prompt1,prompt2,prompt3,prompt4,prompt_live,prompt2,prompt4,prompt1]

def call_endpoint(prompt):
    current_time = datetime.datetime.now()
    formatted_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
    print("start invoke timestamp:", formatted_time)
    response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompt,
                "parameters": parameters
            }
            ),
            ContentType="application/json",
        )

    event_stream = response_model['Body']
    index=0
    for event in event_stream:
        eventJson=event['PayloadPart']['Bytes'].decode('utf-8')
        #output=extract_unicode_chars(eventJson)
        output=(eventJson)
        if index==3:
            first_ouput_time = datetime.datetime.now()
            formatted_time = first_ouput_time.strftime("%Y-%m-%d %H:%M:%S")
            print("first output token:"+output+" timestamp:"+formatted_time)
        index=index+1

    
    #for line in LineIterator(event_stream):
    #        data = json.loads(line)
    #        output += data['outputs'][0]+''
    #        print(output)


#call_endpoint(prompt1)

results = Parallel(n_jobs=10, prefer='threads', verbose=1, )(
    delayed(call_endpoint)(prompt)
    for prompt in prompts
)

In [ ]:
import time
import json
from joblib import Parallel, delayed
prompts = [prompt_live,prompt_live,prompt_live]


def call_endpoint(prompt):
    input = {"inputs": prompt, "parameters": parameters}
    input = json.dumps(input)
    start = time.time()

    response = smr_client.invoke_endpoint(EndpointName=endpoint_name,
                                       ContentType='application/json',
                                       Accept='application/json',
                                       Body=input)
    results = response['Body'].read().decode("utf-8")
    end = time.time()
    process_time=end-start
    print("process time:"+str(int(process_time)))
    ouputJson=json.loads(results)
    print(ouputJson)


results = Parallel(n_jobs=3, prefer='threads', verbose=1)(
    delayed(call_endpoint)(prompt)
    for prompt in prompts
)

## Clean up the environment

In [ ]:
sess.delete_endpoint(endpoint_name)
sess.delete_endpoint_config(endpoint_name)
model.delete_model()